In [1]:
from PIL import Image
import numpy as np
import os
import tqdm
import pandas as pd
import math

In [2]:
root = "\\\\hastur\\hekalo\\Datasets\\CheXpert-v1.0-small\\"

In [3]:
train_csv = os.path.join(root, "labels_raw\\train.csv")
valid_csv = os.path.join(root, "label_raw\\valid.csv")

In [4]:
train_labels = pd.read_csv(train_csv)
valid_labels = pd.read_csv(valid_csv)

FileNotFoundError: [Errno 2] File \\hastur\hekalo\Datasets\CheXpert-v1.0-small\labels_raw\train.csv does not exist: '\\\\hastur\\hekalo\\Datasets\\CheXpert-v1.0-small\\labels_raw\\train.csv'

In [5]:
len(train_labels), len(valid_labels)

NameError: name 'train_labels' is not defined

In [32]:
train_labels

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN
223410,CheXpert-v1.0-small/train/patient64537/study1/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,-1.0,NaN,-1.0,NaN,NaN,NaN
223411,CheXpert-v1.0-small/train/patient64538/study1/...,Female,0,Frontal,AP,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223412,CheXpert-v1.0-small/train/patient64539/study1/...,Female,0,Frontal,AP,NaN,NaN,1.0,1.0,NaN,NaN,NaN,-1.0,1.0,0.0,NaN,NaN,NaN,0.0


In [33]:
train_labels.drop(columns=['Sex', 'Age', 'Frontal/Lateral', 'AP/PA'], inplace=True)
valid_labels.drop(columns=['Sex', 'Age', 'Frontal/Lateral', 'AP/PA'], inplace=True)

In [34]:
train_labels = train_labels[["Path", "Pneumonia"]]
valid_labels = valid_labels[["Path", "Pneumonia"]]

In [35]:
train_labels = train_labels.fillna(0)
train_labels

,Path,Pneumonia
0,CheXpert-v1.0-small/train/patient00001/study1/...,0.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,0.0
2,CheXpert-v1.0-small/train/patient00002/study1/...,0.0
3,CheXpert-v1.0-small/train/patient00002/study1/...,0.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,0.0
...,...,...
223409,CheXpert-v1.0-small/train/patient64537/study2/...,0.0
223410,CheXpert-v1.0-small/train/patient64537/study1/...,0.0
223411,CheXpert-v1.0-small/train/patient64538/study1/...,0.0
223412,CheXpert-v1.0-small/train/patient64539/study1/...,-1.0


In [36]:
valid_labels

,Path,Pneumonia
0,CheXpert-v1.0-small/valid/patient64541/study1/...,0.0
1,CheXpert-v1.0-small/valid/patient64542/study1/...,0.0
2,CheXpert-v1.0-small/valid/patient64542/study1/...,0.0
3,CheXpert-v1.0-small/valid/patient64543/study1/...,0.0
4,CheXpert-v1.0-small/valid/patient64544/study1/...,0.0
...,...,...
229,CheXpert-v1.0-small/valid/patient64736/study1/...,0.0
230,CheXpert-v1.0-small/valid/patient64737/study1/...,0.0
231,CheXpert-v1.0-small/valid/patient64738/study1/...,0.0
232,CheXpert-v1.0-small/valid/patient64739/study1/...,0.0


In [37]:
train_labels["Pneumonia"].value_counts()

 0.0    198605
-1.0     18770
 1.0      6039
Name: Pneumonia, dtype: int64

In [39]:
valid_labels["Pneumonia"].value_counts()

0.0    226
1.0      8
Name: Pneumonia, dtype: int64

In [42]:
destination = os.path.join(root, "labels", "pneumonia")
destination

'\\\\hastur\\scratch\\hekalo\\Datasets\\CheXpert\\labels\\pneumonia'

In [43]:
train_labels.to_csv(os.path.join(destination, "train.csv"))

OSError: [Errno 22] Invalid argument

In [ ]:
valid_labels.to_csv(os.path.join(destination, "valid.csv"))